# 🎯 ЭТАП 1: ОПТИМИЗАЦИЯ MZA КЛАССИФИКАТОРА

## 📅 Дата: 26.10.2025
## 🎯 Цель: Достичь максимального качества разделения рыночных зон

### 📊 **Задачи:**
1. **Оптимизировать динамические веса** для разных уровней волатильности
2. **Настроить пороговые значения** для всех индикаторов MZA
3. **Протестировать на разных таймфреймах** - найти оптимальные параметры
4. **Создать систему автоматической настройки** параметров

### 🏆 **Ожидаемые результаты:**
- Максимальное качество разделения зон (Economic Value > 0.0005)
- Оптимизированные параметры для всех таймфреймов
- Система автоматической настройки параметров
- Готовность к оптимизации индикаторов


## 📦 ИМПОРТЫ И НАСТРОЙКИ


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройки визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Настройки pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("📦 Импорты загружены успешно")
print(f"📅 Дата: {datetime.now().strftime('%d.%m.%Y %H:%M')}")


📦 Импорты загружены успешно
📅 Дата: 27.10.2025 00:25


## 🔧 ЗАГРУЗКА СИСТЕМЫ ОПТИМИЗАЦИИ


In [3]:
# ПРОВЕРКА ПУТЕЙ И ФАЙЛОВ ПЕРЕД ИНИЦИАЛИЗАЦИЕЙ
import sys
from pathlib import Path
import importlib

print("🔍 ПРОВЕРКА ПУТЕЙ И ФАЙЛОВ:")

# Определяем корневой каталог проекта (поднимаемся на 2 уровня выше)
current_dir = Path.cwd()
project_root = current_dir.parent.parent  # Поднимаемся из indicator_optimization/01_mza_optimization в корень

print(f"📁 Текущий рабочий каталог: {current_dir}")
print(f"📁 Корневой каталог проекта: {project_root}")

# Проверяем существование CSV файлов в корневом каталоге
csv_files = ['df_btc_15m.csv', 'df_btc_30m.csv', 'df_btc_1h.csv', 'df_btc_4h.csv', 'df_btc_1d.csv']
print(f"\n📊 Проверка CSV файлов в корневом каталоге ({project_root}):")
csv_exists = True
for file in csv_files:
    file_path = project_root / file
    exists = file_path.exists()
    print(f"   {'✅' if exists else '❌'} {file}: {exists}")
    if not exists:
        csv_exists = False

# Проверяем существование классификаторов в корневом каталоге
classifiers_path = project_root / 'compare_analyze_indicators' / 'classifiers'
print(f"\n🔧 Проверка классификаторов в {classifiers_path}:")
classifier_files = ['mza_classifier_vectorized.py', 'ml_classifier_optimized.py']
classifiers_exist = True
for file in classifier_files:
    file_path = classifiers_path / file
    exists = file_path.exists()
    print(f"   {'✅' if exists else '❌'} {file}: {exists}")
    if not exists:
        classifiers_exist = False

# Проверяем готовность к инициализации
print(f"\n🎯 ГОТОВНОСТЬ К ИНИЦИАЛИЗАЦИИ:")
print(f"   CSV файлы: {'✅ Готовы' if csv_exists else '❌ Не найдены'}")
print(f"   Классификаторы: {'✅ Готовы' if classifiers_exist else '❌ Не найдены'}")

if not csv_exists or not classifiers_exist:
    print(f"\n⚠️ ВНИМАНИЕ: Не все файлы найдены!")
    print(f"   Убедитесь, что вы находитесь в корневом каталоге проекта")
    print(f"   и все необходимые файлы присутствуют")
    print(f"   Ожидаемый корневой каталог: {project_root}")
else:
    print(f"\n✅ ВСЕ ФАЙЛЫ НАЙДЕНЫ! Можно продолжать инициализацию")


🔍 ПРОВЕРКА ПУТЕЙ И ФАЙЛОВ:
📁 Текущий рабочий каталог: c:\Users\XE\Desktop\CRYPTO_2025\indicator_optimization\01_mza_optimization
📁 Корневой каталог проекта: c:\Users\XE\Desktop\CRYPTO_2025

📊 Проверка CSV файлов в корневом каталоге (c:\Users\XE\Desktop\CRYPTO_2025):
   ✅ df_btc_15m.csv: True
   ✅ df_btc_30m.csv: True
   ✅ df_btc_1h.csv: True
   ✅ df_btc_4h.csv: True
   ✅ df_btc_1d.csv: True

🔧 Проверка классификаторов в c:\Users\XE\Desktop\CRYPTO_2025\compare_analyze_indicators\classifiers:
   ✅ mza_classifier_vectorized.py: True
   ✅ ml_classifier_optimized.py: True

🎯 ГОТОВНОСТЬ К ИНИЦИАЛИЗАЦИИ:
   CSV файлы: ✅ Готовы
   Классификаторы: ✅ Готовы

✅ ВСЕ ФАЙЛЫ НАЙДЕНЫ! Можно продолжать инициализацию


In [4]:
# ИНИЦИАЛИЗАЦИЯ СИСТЕМЫ ОПТИМИЗАЦИИ
# Определяем корневой каталог проекта (поднимаемся на 2 уровня выше)
current_dir = Path.cwd()
project_root = current_dir.parent.parent  # Поднимаемся из indicator_optimization/01_mza_optimization в корень

# Добавляем пути к модулям
sys.path.insert(0, str(project_root / 'indicator_optimization'))

# Принудительно перезагружаем модуль для применения изменений
if 'base_optimization_system' in sys.modules:
    importlib.reload(sys.modules['base_optimization_system'])

from base_optimization_system import IndicatorOptimizationSystem

# Создание системы оптимизации с MZA классификатором
# Явно указываем путь к корневому каталогу проекта
mza_system = IndicatorOptimizationSystem(
    classifier_type='mza', 
    data_path=str(project_root)  # Явно указываем корневой каталог
)

print("✅ MZA система оптимизации загружена")
print(f"📊 Доступные таймфреймы: {list(mza_system.data.keys())}")
print(f"🔧 Классификатор: {mza_system.classifier_type}")


🔍 Пути для импорта:
   Project root: c:\Users\XE\Desktop
   Classifiers path: c:\Users\XE\Desktop\compare_analyze_indicators\classifiers
   Trend classifier path: c:\Users\XE\Desktop\indicators\trading_classifier_iziceros\src
🔍 Проверка файлов:
   MZA файл существует: False - c:\Users\XE\Desktop\compare_analyze_indicators\classifiers\mza_classifier_vectorized.py
   ML файл существует: False - c:\Users\XE\Desktop\compare_analyze_indicators\classifiers\ml_classifier_optimized.py
❌ Не удалось импортировать классификаторы: No module named 'mza_classifier_vectorized'
🔄 Пробуем альтернативные способы импорта...
❌ Не удалось загрузить VectorizedMZAClassifier: [Errno 2] No such file or directory: 'c:\\Users\\XE\\Desktop\\compare_analyze_indicators\\classifiers\\mza_classifier_vectorized.py'
❌ Не удалось загрузить OptimizedMarketRegimeMLClassifier: [Errno 2] No such file or directory: 'c:\\Users\\XE\\Desktop\\compare_analyze_indicators\\classifiers\\ml_classifier_optimized.py'
🔧 Инициализация с

## 🎯 ПРАКТИЧЕСКАЯ ОПТИМИЗАЦИЯ ПАРАМЕТРОВ MZA


In [ ]:
# Импорт оптимизатора параметров MZA
from mza_parameter_optimizer import MZAParameterOptimizer

# Создание оптимизатора
optimizer = MZAParameterOptimizer()

print("🎯 MZA Parameter Optimizer загружен")
print(f"📊 Параметров для оптимизации: {len(optimizer.optimizable_params)}")
print(f"🔧 Доступные параметры: {list(optimizer.optimizable_params.keys())}")


## 📊 ТЕСТИРОВАНИЕ НА РАЗНЫХ ТАЙМФРЕЙМАХ


In [ ]:
# Тестирование оптимизации на разных таймфреймах
timeframes = ['15m', '30m', '1h', '4h', '1d']
optimization_results = {}

print("🚀 НАЧИНАЕМ ОПТИМИЗАЦИЮ НА РАЗНЫХ ТАЙМФРЕЙМАХ")
print("=" * 60)

for tf in timeframes:
    if tf in mza_system.data:
        print(f"\n📊 Оптимизация для таймфрейма: {tf}")
        print(f"   📈 Записей: {len(mza_system.data[tf]):,}")
        
        # Используем последние 5000 записей для оптимизации (для скорости)
        data_sample = mza_system.data[tf].tail(5000)
        
        try:
            # Запускаем оптимизацию
            results = optimizer.optimize_parameters(data_sample)
            optimization_results[tf] = results
            
            print(f"   ✅ Оптимизация завершена")
            print(f"   🏆 Лучший Economic Value: {results['best_score']:.6f}")
            print(f"   📊 Протестировано комбинаций: {results['total_tested']}")
            
        except Exception as e:
            print(f"   ❌ Ошибка оптимизации: {e}")
            optimization_results[tf] = {'error': str(e)}
    else:
        print(f"\n❌ Данные для {tf} не найдены")

print(f"\n✅ Оптимизация завершена для {len(optimization_results)} таймфреймов")


## 📄 ЭКСПОРТ ОПТИМИЗИРОВАННЫХ ПАРАМЕТРОВ В PINE SCRIPT


In [ ]:
# Экспорт оптимальных параметров для каждого таймфрейма
print("📄 ЭКСПОРТ ОПТИМИЗИРОВАННЫХ ПАРАМЕТРОВ В PINE SCRIPT")
print("=" * 60)

# Находим лучший результат по всем таймфреймам
best_timeframe = None
best_score = -float('inf')

for tf, results in optimization_results.items():
    if 'error' not in results and results['best_score'] > best_score:
        best_score = results['best_score']
        best_timeframe = tf

print(f"🏆 Лучший результат: {best_timeframe} (Economic Value: {best_score:.6f})")

# Экспортируем параметры для лучшего таймфрейма
if best_timeframe:
    best_results = optimization_results[best_timeframe]
    
    # Создаем Pine Script код
    pine_code = optimizer.export_to_pine_script(f"optimized_mza_{best_timeframe}.pine")
    
    print(f"\n📄 Pine Script код создан для таймфрейма: {best_timeframe}")
    print(f"📁 Файл сохранен: optimized_mza_{best_timeframe}.pine")
    
    # Показываем первые строки кода
    print(f"\n📋 Первые строки Pine Script кода:")
    print("-" * 40)
    lines = pine_code.split('\n')[:20]
    for line in lines:
        print(line)
    print("...")
    print("-" * 40)

# Создаем сводную таблицу результатов
print(f"\n📊 СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
print("=" * 50)
print(f"{'Таймфрейм':<8} {'Economic Value':<15} {'Комбинаций':<12} {'Статус':<10}")
print("-" * 50)

for tf, results in optimization_results.items():
    if 'error' in results:
        print(f"{tf:<8} {'N/A':<15} {'N/A':<12} {'Ошибка':<10}")
    else:
        ev = results['best_score']
        tested = results['total_tested']
        print(f"{tf:<8} {ev:<15.6f} {tested:<12} {'Успех':<10}")

print("-" * 50)
